In [0]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


## Install OpenPose

In [0]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  # install new CMake becaue of CUDA10
  !wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
  !tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local
  # clone openpose
  !git clone -q --depth 1 $git_repo_url
  !sed -i 's/execute_process(COMMAND git checkout master WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/execute_process(COMMAND git checkout f019d0dfe86f49d1140961f8c7dec22130c83154 WORKING_DIRECTORY ${CMAKE_SOURCE_DIR}\/3rdparty\/caffe)/g' openpose/CMakeLists.txt
  # install system dependencies
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  # install python dependencies
  !pip install -q youtube-dl
  # build openpose
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  
from IPython.display import YouTubeVideo

In [0]:
!pwd 

/content


In [0]:
!chmod u+x batch_openpose.sh

In [0]:
!./batch_openpose.sh

In [0]:
# detect poses of images
!cd openpose && ./build/examples/openpose/openpose.bin --image_dir '../drive/My Drive/636/dataset/neg/frames1' --write_json output/ --display 0 --render_pose 0  --face --hand 

Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...

Error:
Folder ../drive/My/ does not exist.

Coming from:
- /content/openpose/src/openpose/utilities/fileSystem.cpp:getFilesOnDirectory():325
- /content/openpose/src/openpose/utilities/fileSystem.cpp:getFilesOnDirectory():379
- /content/openpose/src/openpose/utilities/fileSystem.cpp:getFilesOnDirectory():421
- /content/openpose/src/openpose/producer/imageDirectoryReader.cpp:getImagePathsOnDirectory():23
- /content/openpose/src/openpose/producer/producer.cpp:createProducer():475
- /content/openpose/include/openpose/wrapper/wrapperAuxiliary.hpp:configureThreadManager():1188
- /content/openpose/include/openpose/wrapper/wrapper.hpp:exec():424


In [0]:
# # detect poses of images
# !cd openpose && ./build/examples/openpose/openpose.bin --number_people_max 1 --3d --image_dir '../drive/My Drive/636/dataset/neg/frames1' --write_json output/ --display 0 --render_pose 0  --face --hand 

Tagging:

In [0]:
import json
import pandas as pd
import numpy as np
import os, os.path

In [0]:
cnt_pose = 0
cnt_face = 0
cnt_lh = 0
cnt_rh = 0
with open('./drive/My Drive/636/dataset/pos/1/1_keypoints.json', 'r') as f:
  data = json.load(f)['people'][0]
  for i in data['pose_keypoints_2d']:
    cnt_pose += 1
  for i in data['face_keypoints_2d']:
    cnt_face += 1
  for i in data['hand_left_keypoints_2d']:
    cnt_lh += 1
  for i in data['hand_right_keypoints_2d']:
    cnt_rh += 1

In [0]:
print(cnt_pose, cnt_face, cnt_lh, cnt_rh)

75 210 63 63


In [0]:
COL_NAMES = ['person_id']
for i in range(cnt_pose):
  name = 'p_' + str(i)
  COL_NAMES.append(name)
for i in range(cnt_face):
  name = 'f_' + str(i)
  COL_NAMES.append(name)
for i in range(cnt_lh):
  name = 'lh_' + str(i)
  COL_NAMES.append(name)
for i in range(cnt_rh):
  name = 'rh_' + str(i)
  COL_NAMES.append(name)
COL_NAMES.append('label')

Negative

In [0]:
# first matrix, then dataframe
NEG_PATH = './drive/My Drive/636/dataset/neg/'
row_neg = 0
matrix_neg = []
for i in range(1, 40):
  folderpath = NEG_PATH + str(i) + '/' 
  for name in os.listdir(folderpath):
    filepath = os.path.join(folderpath, name)
    if os.path.isfile(filepath):
      with open(filepath, 'r') as f:
        data = json.load(f)['people']
        if len(data) > 0:
          matrix_neg.append([])
          matrix_neg[row_neg].append(i)
          for j in range(cnt_pose):
            matrix_neg[row_neg].append(float(data[0]['pose_keypoints_2d'][j]))
          for j in range(cnt_face):
            matrix_neg[row_neg].append(float(data[0]['face_keypoints_2d'][j]))
          for j in range(cnt_lh):
            matrix_neg[row_neg].append(float(data[0]['hand_left_keypoints_2d'][j]))
          for j in range(cnt_rh):
            matrix_neg[row_neg].append(float(data[0]['hand_right_keypoints_2d'][j]))
          matrix_neg[row_neg].append(-1)
          row_neg += 1

In [0]:
df_neg = pd.DataFrame(matrix_neg, columns = COL_NAMES)

In [0]:
df_neg.shape

(2688, 413)

In [0]:
del matrix_neg

In [0]:
NEG_PATH = './drive/My Drive/636/dataset/neg/'
row_neg = 0
matrix_neg = []
for i in range(40, 86):
  folderpath = NEG_PATH + str(i) + '/' 
  for name in os.listdir(folderpath):
    filepath = os.path.join(folderpath, name)
    if os.path.isfile(filepath):
      with open(filepath, 'r') as f:
        data = json.load(f)['people']
        if len(data) > 0:
          matrix_neg.append([])
          matrix_neg[row_neg].append(i)
          for j in range(cnt_pose):
            matrix_neg[row_neg].append(float(data[0]['pose_keypoints_2d'][j]))
          for j in range(cnt_face):
            matrix_neg[row_neg].append(float(data[0]['face_keypoints_2d'][j]))
          for j in range(cnt_lh):
            matrix_neg[row_neg].append(float(data[0]['hand_left_keypoints_2d'][j]))
          for j in range(cnt_rh):
            matrix_neg[row_neg].append(float(data[0]['hand_right_keypoints_2d'][j]))
          matrix_neg[row_neg].append(-1)
          row_neg += 1

In [0]:
df_neg2 = pd.DataFrame(matrix_neg, columns = COL_NAMES)

In [0]:
df_neg2.shape

(1487, 413)

In [0]:
df_neg = pd.concat([df_neg, df_neg2], ignore_index=True)

In [0]:
df_neg.shape

(4175, 413)

In [0]:
df_neg.tail()

,person_id,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,p_17,p_18,p_19,p_20,p_21,p_22,p_23,p_24,p_25,p_26,p_27,p_28,p_29,p_30,p_31,p_32,p_33,p_34,p_35,p_36,p_37,p_38,...,rh_24,rh_25,rh_26,rh_27,rh_28,rh_29,rh_30,rh_31,rh_32,rh_33,rh_34,rh_35,rh_36,rh_37,rh_38,rh_39,rh_40,rh_41,rh_42,rh_43,rh_44,rh_45,rh_46,rh_47,rh_48,rh_49,rh_50,rh_51,rh_52,rh_53,rh_54,rh_55,rh_56,rh_57,rh_58,rh_59,rh_60,rh_61,rh_62,label
4170,85,654.863,274.787,0.802982,619.646,509.894,0.535690,398.166,509.836,0.348031,0.0,0.0,0.0,0.0,0.0,0.0,839.004,517.725,0.414678,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4171,85,652.930,274.881,0.812548,621.518,511.850,0.530628,398.168,509.909,0.332508,0.0,0.0,0.0,0.0,0.0,0.0,839.014,519.662,0.419507,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4172,85,651.008,274.848,0.826359,619.579,509.925,0.537220,394.306,509.903,0.335926,0.0,0.0,0.0,0.0,0.0,0.0,838.982,519.653,0.425953,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4173,85,650.993,274.795,0.835295,617.685,511.866,0.542406,396.236,509.909,0.346731,0.0,0.0,0.0,0.0,0.0,0.0,837.079,519.682,0.417218,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4174,85,650.929,274.784,0.847080,617.645,511.848,0.542359,396.217,509.834,0.346706,0.0,0.0,0.0,0.0,0.0,0.0,837.063,519.691,0.418856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [0]:
df_neg.head()

,person_id,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,p_17,p_18,p_19,p_20,p_21,p_22,p_23,p_24,p_25,p_26,p_27,p_28,p_29,p_30,p_31,p_32,p_33,p_34,p_35,p_36,p_37,p_38,...,rh_24,rh_25,rh_26,rh_27,rh_28,rh_29,rh_30,rh_31,rh_32,rh_33,rh_34,rh_35,rh_36,rh_37,rh_38,rh_39,rh_40,rh_41,rh_42,rh_43,rh_44,rh_45,rh_46,rh_47,rh_48,rh_49,rh_50,rh_51,rh_52,rh_53,rh_54,rh_55,rh_56,rh_57,rh_58,rh_59,rh_60,rh_61,rh_62,label
0,1,637.263,253.175,0.795557,631.307,527.507,0.416001,400.204,537.312,0.325623,0.0,0.0,0.0,0.0,0.0,0.0,872.319,511.844,0.400458,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,1,637.237,253.180,0.800757,629.374,527.519,0.407014,400.189,539.253,0.318323,0.0,0.0,0.0,0.0,0.0,0.0,872.337,511.861,0.388536,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,1,637.208,253.202,0.805482,629.352,529.449,0.394687,400.179,539.254,0.326108,0.0,0.0,0.0,0.0,0.0,0.0,872.337,511.853,0.384289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,1,637.178,253.213,0.809286,631.318,529.461,0.396955,400.189,541.202,0.324595,0.0,0.0,0.0,0.0,0.0,0.0,872.324,511.843,0.383505,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,1,637.186,253.235,0.810125,629.376,529.452,0.393593,400.159,541.201,0.323422,0.0,0.0,0.0,0.0,0.0,0.0,872.320,511.848,0.380503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [0]:
df_neg.to_csv('neg.csv', index=False)

Positive

In [0]:
# first matrix, then dataframe
POS_PATH = './drive/My Drive/636/dataset/pos/'
row = 0
matrix = []
for i in range(1, 81):
  folderpath = POS_PATH + str(i) + '/'
  for name in os.listdir(folderpath):
    filepath = os.path.join(folderpath, name)
    if os.path.isfile(filepath):
      with open(filepath, 'r') as f:
        data = json.load(f)['people']
        # print(json.load(f)['people'][0])
        if len(data) > 0:
          matrix.append([])
          matrix[row].append(i)
          for j in range(cnt_pose):
            matrix[row].append(float(data[0]['pose_keypoints_2d'][j]))
          for j in range(cnt_face):
            matrix[row].append(float(data[0]['face_keypoints_2d'][j]))
          for j in range(cnt_lh):
            matrix[row].append(float(data[0]['hand_left_keypoints_2d'][j]))
          for j in range(cnt_rh):
            matrix[row].append(float(data[0]['hand_right_keypoints_2d'][j]))
          matrix[row].append(1)
          row += 1


In [0]:
df_pos = pd.DataFrame(matrix, columns = COL_NAMES)

In [0]:
df_pos.shape

(2606, 413)

In [0]:
df_pos.tail()

,person_id,p_0,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_8,p_9,p_10,p_11,p_12,p_13,p_14,p_15,p_16,p_17,p_18,p_19,p_20,p_21,p_22,p_23,p_24,p_25,p_26,p_27,p_28,p_29,p_30,p_31,p_32,p_33,p_34,p_35,p_36,p_37,p_38,...,rh_24,rh_25,rh_26,rh_27,rh_28,rh_29,rh_30,rh_31,rh_32,rh_33,rh_34,rh_35,rh_36,rh_37,rh_38,rh_39,rh_40,rh_41,rh_42,rh_43,rh_44,rh_45,rh_46,rh_47,rh_48,rh_49,rh_50,rh_51,rh_52,rh_53,rh_54,rh_55,rh_56,rh_57,rh_58,rh_59,rh_60,rh_61,rh_62,label
2601,80,874.573,509.782,0.777790,901.134,909.984,0.368156,480.263,924.677,0.273424,0.0,0.0,0.0,0.0,0.0,0.0,1345.56,880.555,0.252903,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2602,80,868.805,509.669,0.785463,898.164,907.040,0.361534,483.209,927.604,0.249936,0.0,0.0,0.0,0.0,0.0,0.0,1345.55,880.528,0.254539,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2603,80,868.709,506.891,0.799367,895.268,906.990,0.351124,483.198,915.856,0.278367,0.0,0.0,0.0,0.0,0.0,0.0,1333.79,877.560,0.255562,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2604,80,868.723,506.887,0.800780,895.264,907.015,0.348203,483.170,918.789,0.267912,0.0,0.0,0.0,0.0,0.0,0.0,1339.65,880.552,0.250921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2605,80,865.865,506.888,0.803995,895.231,907.002,0.359091,480.276,918.792,0.277047,0.0,0.0,0.0,0.0,0.0,0.0,1330.86,880.549,0.246274,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [0]:
df_pos.to_csv('pos.csv', index=False)

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
def upload(name):
  uploaded = drive.CreateFile({'title': name})
  uploaded.SetContentFile(name)
  uploaded.Upload()
  print('Uploaded file with ID {}'.format(uploaded.get('id')))

In [0]:
upload('pos.csv')

Uploaded file with ID 1wvqEEpezdoZ37jyBd7kZ0nGuyy5LZZxJ


In [0]:
upload('neg.csv')

Uploaded file with ID 15qKq_aCyhnMaxchLvSnoD6NOr2NDLlk-
